In [ ]:
#@markdown <center><h1>Install</h1></center>

%cd /content
!git clone -b totoro5 https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro https://github.com/LucipherDev/ComfyUI-GGUF /content/TotoroUI/custom_nodes/TotoroUI-GGUF
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!pip install -q git+https://github.com/LucipherDev/controlnet_aux.git
!pip install -q -r /content/TotoroUI/custom_nodes/TotoroUI-GGUF/requirements.txt
!apt -y install -qq aria2

import nodes

if not nodes.load_custom_node("custom_nodes/TotoroUI-GGUF"):
  raise Exception("Failed to load GGUF custom node")

In [ ]:
#@markdown <center><h1>Load Models</h1></center>

import torch
from nodes import NODE_CLASS_MAPPINGS

DualCLIPLoaderGGUF = NODE_CLASS_MAPPINGS["DualCLIPLoaderGGUF"]()
UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
ControlNetLoader = NODE_CLASS_MAPPINGS["ControlNetLoader"]()

system_ram = "low ram (Q2_K)" # @param ["low ram (Q2_K)", "high ram (Q8_0)"]

if system_ram == "low ram (Q2_K)":
  print(f"Downloading Flux1-dev-Q2_K...")
  !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q2_K.gguf -d /content/TotoroUI/models/unet -o flux1-dev.gguf
elif system_ram == "high ram (Q8_0)":
  print(f"Downloading Flux1-dev-Q8_0...")
  !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf -d /content/TotoroUI/models/unet -o flux1-dev.gguf

print("Downloading VAE...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft

print("Downloading Clips...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q3_K_L.gguf -d /content/TotoroUI/models/clip -o t5-v1_1-xxl-encoder-Q3_K_L.gguf

print("Downloading Controlnet Preprocessors...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/Kijai/DepthAnythingV2-safetensors/resolve/main/depth_anything_v2_vitl_fp16.safetensors -d /content/TotoroUI/models/controlnet_preprocessors -o depth_anything_v2_vitl_fp16.safetensors
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/ControlNetHED.pth -d /content/TotoroUI/models/controlnet_preprocessors -o ControlNetHED.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/body_pose_model.pth -d /content/TotoroUI/models/controlnet_preprocessors -o body_pose_model.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/hand_pose_model.pth -d /content/TotoroUI/models/controlnet_preprocessors -o hand_pose_model.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/facenet.pth -d /content/TotoroUI/models/controlnet_preprocessors -o facenet.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model.pth -d /content/TotoroUI/models/controlnet_preprocessors -o sk_model.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model2.pth -d /content/TotoroUI/models/controlnet_preprocessors -o sk_model2.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/fal/teed/resolve/main/5_model.pth -d /content/TotoroUI/models/controlnet_preprocessors -o 5_model.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/table5_pidinet.pth -d /content/TotoroUI/models/controlnet_preprocessors -o table5_pidinet.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/scannet.pt -d /content/TotoroUI/models/controlnet_preprocessors -o scannet.pt
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/mlsd_large_512_fp32.pth -d /content/TotoroUI/models/controlnet_preprocessors -omlsd_large_512_fp32.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/netG.pth -d /content/TotoroUI/models/controlnet_preprocessors -o netG.pth
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/ybelkada/segment-anything/resolve/main/checkpoints/sam_vit_b_01ec64.pth -d /content/TotoroUI/models/controlnet_preprocessors -o sam_vit_b.pth

print("Downloading Controlnet Union...")
!aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M https://huggingface.co/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro/resolve/main/diffusion_pytorch_model.safetensors -d /content/TotoroUI/models/controlnet -o FLUX.1-dev-Controlnet-Union-Pro.safetensors

with torch.inference_mode():
    print("Loading Clips...")
    clip = DualCLIPLoaderGGUF.load_clip("t5-v1_1-xxl-encoder-Q3_K_L.gguf", "clip_l.safetensors", "flux")[0]
    print("Loading VAE...")
    vae = VAELoader.load_vae("ae.sft")[0]
    print(f"Loading Flux.1-dev...")
    unet = UnetLoaderGGUF.load_unet(f"flux1-dev.gguf")[0]
    print("Loading Controlnet Union...")
    controlnet = ControlNetLoader.load_controlnet("FLUX.1-dev-Controlnet-Union-Pro.safetensors")[0]

    unet_f, clip_f = unet, clip

print("All Models Loaded!")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import re
import os
import gc
import random
import numpy as np
from PIL import Image
from google.colab import files

import nodes
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro_extras import nodes_flux
from totoro_extras import nodes_controlnet
from totoro import model_management

from controlnet_aux import CannyDetector, DepthAnythingDetector, HEDdetector, OpenposeDetector, PidiNetDetector, TEEDdetector, LineartDetector, LineartAnimeDetector, MLSDdetector, NormalBaeDetector, SamDetector

CLIPTextEncodeFlux = nodes_flux.NODE_CLASS_MAPPINGS["CLIPTextEncodeFlux"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()
SetUnionControlNetType = nodes_controlnet.NODE_CLASS_MAPPINGS["SetUnionControlNetType"]()
ControlNetApplyAdvanced = NODE_CLASS_MAPPINGS["ControlNetApplyAdvanced"]()

In [ ]:
# @markdown <center><h1>Functions</h1></center>

loras = {
    "xlabs_flux_anime":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/anime_lora_comfy_converted.safetensors",
         "filename": "xlabs_anime_lora.safetensors",
         "triggers": "anime"
         },
    "xlabs_flux_art":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/art_lora_comfy_converted.safetensors",
         "filename": "xlabs_art_lora.safetensors",
         "triggers": "art"
         },
    "xlabs_flux_disney":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/disney_lora_comfy_converted.safetensors",
         "filename": "xlabs_disney_lora.safetensors",
         "triggers": "disney style"
         },
    "xlabs_flux_mjv6":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/mjv6_lora_comfy_converted.safetensors",
         "filename": "xlabs_mjv6_lora.safetensors"
         },
    "xlabs_flux_realism":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/realism_lora_comfy_converted.safetensors",
         "filename": "xlabs_realism_lora.safetensors"
         },
    "xlabs_flux_scenery":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/scenery_lora_comfy_converted.safetensors",
         "filename": "xlabs_scenery_lora.safetensors",
         "triggers": "scenery style"
         },
    "xlabs_flux_furry":
     {
         "url": "https://huggingface.co/XLabs-AI/flux-lora-collection/resolve/main/furry_lora.safetensors",
        "filename": "xlabs_flux_furry_lora.safetensors"
        }
}


def load_loras(prompt):
  # @markdown <ul><li><h2>Load Loras</h2>- Add &lt;lora_name:model_strength&gt; to Prompt</li></ul>

  global unet, clip, unet_f, clip_f

  unet_f, clip_f = unet, clip

  matches = re.findall(r"<\s*([^:]+?)\s*:\s*([0-9.]+)\s*>", prompt)

  loras_list = [(name.strip(), float(value)) for name, value in matches]

  if len(loras_list):
    print("Loading Loras...")

  for lora_tuple in loras_list:
    lora = loras.get(lora_tuple[0], None)

    if lora:
      !aria2c --quiet --console-log-level=error --auto-file-renaming=false --allow-overwrite=false -c -x 16 -s 16 -k 1M {lora["url"]} -d /content/TotoroUI/models/loras -o {lora["filename"]}

      with torch.inference_mode():
        unet_f, clip_f = LoraLoader.load_lora(unet_f, clip_f, lora["filename"], lora_tuple[1], lora_tuple[1])

      print(f"Loaded Lora: {lora_tuple[0]}")
    else:
      print(f"Lora not listed: {lora_tuple[0]}")

def clean_prompt(prompt):
  cleaned_prompt = re.sub(r"<.*?>", "", prompt)

  return cleaned_prompt

def cuda_gc():
  try:
    model_management.soft_empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
  except:
    pass

def img_tensor_to_np(img_tensor):
  img_tensor = img_tensor.clone() * 255.0
  return img_tensor.squeeze().numpy().astype(np.uint8)

def img_np_to_tensor(img_np_list):
  return torch.from_numpy(img_np_list.astype(np.float32) / 255.0).unsqueeze(0)

def controlnet_preprocess(width, height):
  # @markdown <ul><li><h2>Controlnet Preprocessor</h2></li></ul>

  process_image = True # @param {"type":"boolean"}
  preprocessor_type = "depth" #  @param ["none", "openpose", "depth", "hed", "pidi", "scribble", "ted", "canny", "lineart", "anime_lineart", "mlsd", "normal", "segment"]
  input_image = "/content/test.png" # @param {"type":"string"}
  resolution = 512 # @param {"type":"slider","min":512,"max":2048,"step":1}
  resize_mode = "Just Resize" # @param ["Just Resize","Resize and Fill","Crop and Resize"]

  image_input = LoadImage.load_image(input_image)[0]

  if process_image:
    image_np = img_tensor_to_np(image_input)
    img = Image.fromarray(image_np)

    if resize_mode == "Just Resize":
        img = img.resize((width, height), Image.Resampling.LANCZOS)

    elif resize_mode == "Resize and Fill":
        img.thumbnail((width, height), Image.ANTIALIAS)
        result = Image.new("RGB", (width, height), (0, 0, 0))
        offset = ((width - img.width) // 2, (height - img.height) // 2)
        result.paste(img, offset)
        img = result

    elif resize_mode == "Crop and Resize":
        src_width, src_height = img.size
        src_aspect = src_width / src_height
        target_aspect = width / height

        if src_aspect > target_aspect:
            new_width = int(src_height * target_aspect)
            left = (src_width - new_width) // 2
            img = img.crop((left, 0, left + new_width, src_height))
        else:
            new_height = int(src_width / target_aspect)
            top = (src_height - new_height) // 2
            img = img.crop((0, top, src_width, top + new_height))

        img = img.resize((width, height), Image.Resampling.LANCZOS)

    image_np = np.array(img).astype(np.uint8)

    if preprocessor_type == "none":
      processed_image = Image.fromarray(image_np)

    if preprocessor_type == "canny":
      canny = CannyDetector()
      processed_image = canny(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "depth":
      depth = DepthAnythingDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="depth_anything_v2_vitl_fp16.safetensors")
      processed_image = depth(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "hed":
      hed = HEDdetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="ControlNetHED.pth")
      processed_image = hed(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "openpose":
      openpose = OpenposeDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="body_pose_model.pth", hand_filename="hand_pose_model.pth", face_filename="facenet.pth")
      processed_image = openpose(image_np, detect_resolution=resolution, image_resolution=resolution, include_hand=True, include_face=True)

    elif preprocessor_type == "pidi":
      pidi = PidiNetDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="table5_pidinet.pth")
      processed_image = pidi(image_np, detect_resolution=resolution, image_resolution=resolution, safe=True)

    elif preprocessor_type == "scribble":
      scribble = HEDdetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="ControlNetHED.pth")
      processed_image = scribble(image_np, detect_resolution=resolution, image_resolution=resolution, scribble=True)

    elif preprocessor_type == "ted":
      ted = TEEDdetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="5_model.pth")
      processed_image = ted(image_np, detect_resolution=resolution)

    elif preprocessor_type == "lineart":
      lineart = LineartDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="sk_model.pth", coarse_filename="sk_model2.pth")
      processed_image = lineart(image_np, detect_resolution=resolution, image_resolution=resolution, coarse=True)

    elif preprocessor_type == "anime_lineart":
      anime_lineart = LineartAnimeDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="netG.pth")
      processed_image = anime_lineart(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "mlsd":
      mlsd = MLSDdetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="mlsd_large_512_fp32.pth")
      processed_image = mlsd(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "normal":
      normal = NormalBaeDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", filename="scannet.pt")
      processed_image = normal(image_np, detect_resolution=resolution, image_resolution=resolution)

    elif preprocessor_type == "segment":
      segment = SamDetector.from_pretrained("/content/TotoroUI/models/controlnet_preprocessors", model_type="vit_b", filename="sam_vit_b.pth")
      processed_image = segment(image_np, detect_resolution=resolution, image_resolution=resolution)

    processed_image_np = np.array(processed_image)
    processed_image_tensor = img_np_to_tensor(processed_image_np)

    display(Image.fromarray(processed_image_np))

    return processed_image_tensor

  else:
    return image_input

def apply_controlnet(cond, width, height):
  # @markdown <ul><li><h2>Union Controlnet</h2></li></ul>

  controlnet_type = "depth" # @param ["openpose", "depth", "hed/pidi/scribble/ted", "canny/lineart/anime_lineart/mlsd", "normal", "segment", "tile", "repaint" ]
  strength = 0.5 # @param {"type":"slider","min":0.0,"max":10.0,"step":0.01}
  start_percent = 0 # @param {"type":"slider","min":0.0,"max":1.0,"step":0.001}
  end_percent = 0.225 # @param {"type":"slider","min":0.0,"max":1.0,"step":0.001}

  cond_neg = CLIPTextEncodeFlux.encode(clip_f, "", "", 0)[0]

  print("Loading Controlnet...")

  preprocessed_image = controlnet_preprocess(width, height)

  controlnet_f = SetUnionControlNetType.set_controlnet_type(controlnet, controlnet_type)[0]

  cond = ControlNetApplyAdvanced.apply_controlnet(cond, cond_neg, controlnet_f, preprocessed_image, strength, start_percent, end_percent, vae)[0]

  del controlnet_f

  return cond

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

def save_image(decoded, path, name, download=False):
  full_path = os.path.abspath(os.path.join(path, name))
  Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save( full_path)

  img = Image.open(full_path)
  display(img)

  if download:
    files.download(full_path)

@torch.inference_mode()
def generate(prompt, width, height, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download, mode="t2i", input_img=None, denoise=1.0):
  global unet, clip, unet_f, clip_f

  print("Prompt Received")

  load_loras(prompt)
  prompt = clean_prompt(prompt)

  if mode == "t2i":
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

  elif mode == "i2i":
    image = LoadImage.load_image(input_img)[0]
    latent_image = ImageScaleToTotalPixels.upscale(image, "lanczos", 1.0)[0]
    latent_image = VAEEncode.encode(vae, latent_image)[0]

  cond = CLIPTextEncodeFlux.encode(clip_f, prompt, prompt, guidance)[0]

  cond = apply_controlnet(cond, width, height)

  guider = BasicGuider.get_guider(unet_f, cond)[0]
  sampler = KSamplerSelect.get_sampler(sampler_name)[0]
  sigmas = BasicScheduler.get_sigmas(unet_f, scheduler, steps, denoise)[0]

  for i in range(0, batch_size):
    if fixed_seed == 0:
      seed = random.randint(0, 18446744073709551615)
    else:
      seed = fixed_seed

    print("Seed:", seed)

    noise = RandomNoise.get_noise(seed)[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    save_image(decoded, "/content", f"flux_{mode}_{seed}_{i}.png", auto_download)

  cuda_gc()

print(f"{'Lora Name':<40} {'Trigger Words':<40}")
print("-" * 80)

for key_name, details in loras.items():
    trigger_words = details.get("triggers", "N/A")
    print(f"{key_name:<40} {trigger_words:<40}")

In [ ]:
#@markdown <center><h1>Txt2Img</h1></center>

positive_prompt = "" # @param {"type":"string"}
width = 512 # @param {"type":"slider","min":256,"max":2048,"step":1}
height = 512 # @param {"type":"slider","min":256,"max":2048,"step":1}
fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
guidance = 4.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
steps = 25 # @param {"type":"slider","min":4,"max":50,"step":1}
sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
auto_download = False # @param {"type":"boolean"}

generate(positive_prompt, width, height, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download)

In [ ]:
#@markdown <center><h1>Img2Img</h1></center>

positive_prompt = "anime style" # @param {"type":"string"}
fixed_seed = 0 # @param {"type":"slider","min":0,"max":18446744073709552000,"step":1}
guidance = 4.5 # @param {"type":"slider","min":0,"max":20,"step":0.5}
steps = 25 # @param {"type":"slider","min":4,"max":50,"step":1}
sampler_name = "euler" # @param ["euler","heun","heunpp2","heunpp2","dpm_2","lms","dpmpp_2m","ipndm","deis","ddim","uni_pc","uni_pc_bh2"]
scheduler = "simple" # @param ["normal","sgm_uniform","simple","ddim_uniform"]
input_img = "/content/test.png" # @param {"type":"string"}
denoise = 0.85 # @param {"type":"slider","min":0,"max":1,"step":0.01}
batch_size = 1 # @param {"type":"slider","min":1,"max":20,"step":1}
auto_download = False # @param {"type":"boolean"}


generate(positive_prompt, 0, 0, fixed_seed, guidance, steps, sampler_name, scheduler, batch_size, auto_download, "i2i", input_img, denoise)